# Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
nltk.download('wordnet')
nltk.download('punkt')
stop_words = stopwords.words('english')
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import re
from fuzzywuzzy import fuzz

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ankit\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ankit\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Importing Feature_1

In [3]:
data = pd.read_csv('FEATURES_III.csv')
data.head(3)

,question1,question2,is_duplicate,norm_wmd,cosine_distance,cityblock_distance,jaccard_distance,canberra_distance,euclidean_distance,minkowski_distance,braycurtis_distance,skew_q1vec,skew_q2vec,kur_q1vec,kur_q2vec
0,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0,0.217555,0.037908,3.774843,1.0,75.949318,0.275348,0.125323,0.137314,0.008893,-0.099771,0.108845,0.344742
1,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0,1.368796,0.574596,15.130415,1.0,190.766894,1.072004,0.482108,0.648993,0.027151,0.060190,0.310524,0.033802
2,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0,0.639209,0.215223,8.840496,1.0,135.849174,0.656084,0.305829,0.332821,0.247069,0.152550,0.042899,-0.489378


# Function for Cleaning Dataset

In [5]:
def cleanText(text):
    #REMOVING PUNTUATIONS AND NUMBERS
    import string
    text1 = "".join([char.lower() for char in text if char not in string.punctuation])
    text2 = re.sub('[0-9]+', "", text1)
    #TOKENIZATION
    text3 = re.split('\W+', text2)
    #STOPWORDS
    text4 = [word for word in text3 if word not in stop_words]
    #lemitization
    lemma  = WordNetLemmatizer()
    text5 = [lemma.lemmatize(word) for word in text4]
    return text5

# Function For Basic Fetautre Extraction

In [6]:
def features(data):
    data['len_q1'] = data.question1.apply(lambda x: len(str(x)))
    data['len_q2'] = data.question2.apply(lambda x: len(str(x)))
    data['diff_len'] = data.len_q1 - data.len_q2
    data['len_char_q1'] = data.question1.apply(lambda x: len(''.join(set(str(x).replace(' ', '')))))
    data['len_char_q2'] = data.question2.apply(lambda x: len(''.join(set(str(x).replace(' ', '')))))
    data['len_word_q1'] = data.question1.apply(lambda x: len(str(x).split()))
    data['len_word_q2'] = data.question2.apply(lambda x: len(str(x).split()))
    data['common_words'] = data.apply(lambda x: len(set(str(x['question1']).lower().split()).intersection(set(str(x['question2']).lower().split()))), axis=1)
    data['fuzz_qratio'] = data.apply(lambda x: fuzz.QRatio(str(x['question1']), str(x['question2'])), axis=1)
    data['fuzz_WRatio'] = data.apply(lambda x: fuzz.WRatio(str(x['question1']), str(x['question2'])), axis=1)
    data['fuzz_partial_ratio'] = data.apply(lambda x: fuzz.partial_ratio(str(x['question1']), str(x['question2'])), axis=1)
    data['fuzz_partial_token_set_ratio'] = data.apply(lambda x: fuzz.partial_token_set_ratio(str(x['question1']), str(x['question2'])), axis=1)
    data['fuzz_partial_token_sort_ratio'] = data.apply(lambda x: fuzz.partial_token_sort_ratio(str(x['question1']), str(x['question2'])), axis=1)
    data['fuzz_token_set_ratio'] = data.apply(lambda x: fuzz.token_set_ratio(str(x['question1']), str(x['question2'])), axis=1)
    data['fuzz_token_sort_ratio'] = data.apply(lambda x: fuzz.token_sort_ratio(str(x['question1']), str(x['question2'])), axis=1)
    return data

In [7]:
#Cleaning dataset
data['question1'] = data['question1'].apply(lambda x:cleanText(x))
data['question2'] = data['question2'].apply(lambda x:cleanText(x))

In [9]:
data.head(2)

,question1,question2,is_duplicate,norm_wmd,cosine_distance,cityblock_distance,jaccard_distance,canberra_distance,euclidean_distance,minkowski_distance,braycurtis_distance,skew_q1vec,skew_q2vec,kur_q1vec,kur_q2vec
0,"[step, step, guide, invest, share, market, india]","[step, step, guide, invest, share, market]",0,0.217555,0.037908,3.774843,1.0,75.949318,0.275348,0.125323,0.137314,0.008893,-0.099771,0.108845,0.344742
1,"[story, kohinoor, kohinoor, diamond]","[would, happen, indian, government, stole, koh...",0,1.368796,0.574596,15.130415,1.0,190.766894,1.072004,0.482108,0.648993,0.027151,0.060190,0.310524,0.033802


In [10]:
#Basic Features Extraction
data = features(data)

In [13]:
data.head(5)

,question1,question2,is_duplicate,norm_wmd,cosine_distance,cityblock_distance,jaccard_distance,canberra_distance,euclidean_distance,minkowski_distance,...,len_word_q1,len_word_q2,common_words,fuzz_qratio,fuzz_WRatio,fuzz_partial_ratio,fuzz_partial_token_set_ratio,fuzz_partial_token_sort_ratio,fuzz_token_set_ratio,fuzz_token_sort_ratio
0,"[step, step, guide, invest, share, market, india]","[step, step, guide, invest, share, market]",0,0.217555,0.037908,3.774843,1.0,75.949318,0.275348,0.125323,...,7,6,5,92,95,98,100,91,100,92
1,"[story, kohinoor, kohinoor, diamond]","[would, happen, indian, government, stole, koh...",0,1.368796,0.574596,15.130415,1.0,190.766894,1.072004,0.482108,...,4,9,1,58,86,91,100,84,84,59
2,"[increase, speed, internet, connection, using,...","[internet, speed, increased, hacking, dns]",0,0.639209,0.215223,8.840496,1.0,135.849174,0.656084,0.305829,...,6,5,1,64,66,67,100,78,70,70
3,"[mentally, lonely, solve]","[find, remainder, mathmath, divided, ]",0,1.263719,0.635976,15.828719,1.0,190.804061,1.127809,0.501902,...,3,5,0,39,39,48,33,33,27,27
4,"[one, dissolve, water, quikly, sugar, salt, me...","[fish, would, survive, salt, water]",0,1.240908,0.332839,11.447098,1.0,154.884094,0.815891,0.364076,...,10,5,1,48,86,62,100,55,51,40


# Saving Complete Features

In [14]:
data.to_csv('completeFeatures.csv', index=False)